In [1]:
# Import the necessary packages

In [33]:
import nltk
import numpy as np
import random
import string
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout,Embedding,LSTM,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [34]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [3]:
# Load the json file 

In [4]:
with open('intents.json') as file:
    data=json.load(file)
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana']},
  {'tag': 'time',
   'patterns': ['what is the best time to visit your office',
    'Your office worki

#### JavaScript Object Notation (JSON) is a standard text-based format for representing structured data based on JavaScript object syntax. It is commonly used for transmitting data in web applications (e.g., sending some data from the server to the client, so it can be displayed on a web page, or vice versa)

In [5]:
trainining_sent=[]
training_labels=[]
labels=[]
responses=[]

In [6]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        trainining_sent.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes=len(labels)
num_classes    

11

In [7]:
print(trainining_sent,end=" ")

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'What are you?', 'Who you are?', 'what is your name', 'what should I call you', 'whats your name?', 'what is the best time to visit your office', 'Your office working time', 'is it open your office in the afternoon', 'Could you help me?', 'give me a hand please', 'Can you help?', 'What can you do for me?', 'I need a support', 'I need a help', 'support me please', 'How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered', 'I need to create a new account', 'how to open a new account', 'I want to create an account', 'can you create an account for me', 'how to open a new account', 'have a complaint', 'I want to raise a complaint', 'there is a complaint about a service'] 

In [27]:
len(trainining_sent)

41

In [8]:
print(training_labels,end="")

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'name', 'name', 'name', 'time', 'time', 'time', 'help', 'help', 'help', 'help', 'help', 'help', 'help', 'options', 'options', 'options', 'options', 'options', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'createaccount', 'complaint', 'complaint', 'complaint']

In [9]:
print(responses,end='')

[['Hello', 'Hi', 'Hi there'], ['See you later', 'Have a nice day', 'Bye! Come back again'], ['Happy to help!', 'Any time!', 'My pleasure', "You're most welcome!"], ['I.m Joana, your bot assistant', "I'm Joana, an Artificial Intelligent bot"], ['You can call me Joana.', "I'm Joana!", 'Just call me as Joana'], ['You can visit our office from Monday to Saturday at any time between 9.00 AM to 5.30 PM'], ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], ['Tell me how can assist you', 'Tell me your problem to assist you', 'Yes Sure, How can I support you'], ['I can help you to solve your problems related to our services'], ['You can just easily create a new account from our web site', 'Just go to our web site and follow the guidelines to create a new account'], ['Please provide us your complaint in order to assist you', 'Please mention your complaint, we will reach you and sorry for any inconvenience caused']]

In [10]:
from nltk.corpus import stopwords
st=stopwords.words('english')

In [11]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [12]:
# Convert text data into vector format

In [32]:
vocab_size = 1000
embedding_dim = 16     # features consider
max_len = 20
oov_token = "<OOV>"   


tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(trainining_sent)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(trainining_sent)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [25]:
print(tokenizer,end=" ")

In [31]:
print(sequences)

[[35], [36], [5, 37, 24], [38], [39], [40], [41, 2, 42], [43], [25], [44, 2], [45, 26], [25, 14, 15, 6], [27, 16, 2], [4, 16, 2], [27, 2, 16], [4, 5, 10, 28], [4, 46, 7, 47, 2], [48, 10, 28], [4, 5, 15, 49, 29, 8, 50, 10, 17], [10, 17, 51, 29], [5, 52, 18, 10, 17, 53, 15, 54], [30, 2, 6, 9], [55, 9, 3, 56, 31], [11, 2, 6], [4, 11, 2, 32, 14, 9], [7, 19, 3, 20], [7, 19, 3, 6], [20, 9, 31], [13, 2, 30, 6, 9], [4, 2, 11, 32], [4, 6, 2, 57], [13, 2, 11, 58, 26], [4, 20, 5, 59], [7, 19, 8, 21, 3, 22, 12], [13, 8, 18, 3, 22, 12], [7, 33, 8, 21, 34, 12], [11, 2, 21, 34, 12, 14, 9], [13, 8, 18, 3, 22, 12], [60, 3, 23], [7, 33, 8, 61, 3, 23], [24, 5, 3, 23, 62, 3, 63]]


In [24]:
print(word_index,end=" ")|

{'<OOV>': 1, 'you': 2, 'a': 3, 'what': 4, 'is': 5, 'help': 6, 'i': 7, 'to': 8, 'me': 9, 'your': 10, 'can': 11, 'account': 12, 'how': 13, 'for': 14, 'the': 15, 'are': 16, 'office': 17, 'open': 18, 'need': 19, 'support': 20, 'create': 21, 'new': 22, 'complaint': 23, 'there': 24, 'thanks': 25, 'helpful': 26, 'who': 27, 'name': 28, 'time': 29, 'could': 30, 'please': 31, 'do': 32, 'want': 33, 'an': 34, 'hi': 35, 'hey': 36, 'anyone': 37, 'hello': 38, 'hay': 39, 'bye': 40, 'see': 41, 'later': 42, 'goodbye': 43, 'thank': 44, "that's": 45, 'should': 46, 'call': 47, 'whats': 48, 'best': 49, 'visit': 50, 'working': 51, 'it': 52, 'in': 53, 'afternoon': 54, 'give': 55, 'hand': 56, 'provide': 57, 'be': 58, 'offered': 59, 'have': 60, 'raise': 61, 'about': 62, 'service': 63} 

In [26]:
print(padded_sequences,end=" ")

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 35]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 36]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5 37 24]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 39]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 40]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 41  2 42]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 43]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 25]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 44  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 45 26]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 25 14 15  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27 16  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4 16  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 27  2 16]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  4  5

In [30]:
len(padded_sequences),len(trainining_sent)

(41, 41)

In [14]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()
enc.fit(training_labels)
training_labels=enc.transform(training_labels)

In [15]:
# Model 

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 11)                187       
Total params: 16,731
Trainable params: 16,731
Non-trainable params: 0
_________________________________________________________________


In [17]:
his=model.fit(padded_sequences,np.array(training_labels),epochs=750)

Epoch 1/750
2/2 [==============================] - 2s 6ms/step - loss: 2.3973 - accuracy: 0.1334
Epoch 2/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3951 - accuracy: 0.0917
Epoch 3/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3925 - accuracy: 0.1067
Epoch 4/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3914 - accuracy: 0.2193
Epoch 5/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3898 - accuracy: 0.1659
Epoch 6/750
2/2 [==============================] - 0s 7ms/step - loss: 2.3882 - accuracy: 0.1659
Epoch 7/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3868 - accuracy: 0.1659
Epoch 8/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3854 - accuracy: 0.1763
Epoch 9/750
2/2 [==============================] - 0s 4ms/step - loss: 2.3842 - accuracy: 0.1555
Epoch 10/750
2/2 [==============================] - 0s 5ms/step - loss: 2.3812 - accuracy: 0.1763
Epoch 11/750
2/2 [===========

In [18]:
model.save('chat_model')

INFO:tensorflow:Assets written to: chat_model_2\assets


In [19]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(enc, ecn_file)

In [22]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder



import random
import pickle


with open("intents.json") as file:
    data = json.load(file)




def chat():
    
    # load trained model
    model = keras.models.load_model('chat_model')


    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)


    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)


    
    max_len = 20
    
    while True:
        print("User:",end="")
        inp = input()
        if inp.lower() == "quit":
            break


        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])


        for i in data['intents']:
            if i['tag'] == tag:
                print( "ChatBot:"  , np.random.choice(i['responses']))




print("Start messaging with the bot (type quit to stop)!")
chat()


Start messaging with the bot (type quit to stop)!
User:


 hi


ChatBot: Hello
User:


 who are you


ChatBot: I'm Joana, an Artificial Intelligent bot
User:


...

 what is the best time to visit your company


ChatBot: You can visit our office from Monday to Saturday at any time between 9.00 AM to 5.30 PM
User:


 thank you


ChatBot: Any time!
User:


 quit
